## Init TF

In [1]:
import numpy as np
import tensorflow as tf
#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
from utils import *
import tensorflow_probability as tfp
import time
# from numba import cuda #gpu library used to clear gpu memory after each trial

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

Autosaving every 180 seconds


## Using NumPy

In [12]:
## from vedo import *
from utils import *
import numpy as np
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti

settings.embedWindow(backend='ipyvtk', verbose = True) #was this

try:
    plt1.closeWindow()
    print("closed")
except:
    pass

plt1 = Plotter(N=1, axes=1, bg = (0.1,0.1,0.1), bg2 = (0.3,0.3,0.3),  interactive=True)
# settings.useParallelProjection = True #makes view orthographic

## uncomment to use VOLPE dataset -----------------------------------------------------
# location = 'C:/Users/Derm/2021-03-10-16-43-50_Velodyne-VLP-16-Data_garminSignage.txt'
# cloud = np.loadtxt(open(location, "rb"), delimiter=",")
# cloud = cloud[~np.isnan(cloud).any(axis=1)] #remove all rows with NaN elements
## ------------------------------------------------------------------------------------

## uncomment to use KITTI dataset -----------------------------------------------------
basedir = 'C:/kitti/'
date = '2011_09_26'
drive = '0005'
frame_range = range(150, 151, 1)
dataset = pykitti.raw(basedir, date, drive)
velo1 = dataset.get_velo(0) # Each scan is a Nx4 array of [x,y,z,reflectance]
cloud = velo1[:,:3]
print(type(cloud))
## ------------------------------------------------------------------------------------

# make 2D sinusioal motion (for debug) ------------------------------------------------
# cloud = np.random.randn(10000,3)
# cloud[:,0] += -50*cloud[:,1] + np.random.randn()*5
# cloud[:,1] += 5
# cloud[:,1] = cloud[:,1] * 3 + 10*np.random.randn() + np.sin(cloud[:,1]*5)*10
# cloud[:,2] += 0.5*cloud[:,1] - 10
##-------------------------------------------------------------------------------------

# f =np.array([200,200,40]) #fidelity in x, y, z #takes ~30s on my dsektop
# lim = np.array([-50,50,-50,50,-10,10])

f =np.array([50,50,1]) #fidelity in x, y, z # < 5s
lim = np.array([-100,100,-100,100,-10,10])

for _ in range(1):
    E = subdivide_scan(cloud,plt1, bounds = lim, fid = f, draw_grid = False, show_pc = True) 
ViewInteractiveWidget(plt1.window)

<class 'numpy.ndarray'>
took 3.1033310890197754 seconds with numpy


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

## Using TensorFlow

In [148]:
## from vedo import *
from utils import *
import numpy as np
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti

#bad vibes
# dev = cuda.get_current_device()
# dev.reset()

settings.embedWindow(backend='ipyvtk', verbose = True) #was this

plt2 = Plotter(N=1, axes=1, bg = (0.1,0.1,0.1), bg2 = (0.3,0.3,0.3),  interactive=True)

## uncomment to use KITTI dataset -----------------------------------------------------
basedir = 'C:/kitti/'
date = '2011_09_26'
drive = '0005'
frame_range = range(150, 151, 1)
dataset = pykitti.raw(basedir, date, drive)
velo1 = dataset.get_velo(0) # Each scan is a Nx4 array of [x,y,z,reflectance]
cloud = velo1[:,:3]
cloud_tensor = tf.convert_to_tensor(cloud, np.float32)
# print(tf.shape(cloud))
## ------------------------------------------------------------------------------------

# # make 2D sinusioal motion (for debug) ------------------------------------------------
# cloud = np.random.randn(10000,3)
# cloud[:,0] += -50*cloud[:,1] + np.random.randn()*5
# cloud[:,1] += 5
# cloud[:,1] = cloud[:,1] * 3 + 10*np.random.randn() + np.sin(cloud[:,1]*5)*10
# cloud[:,2] += 0.5*cloud[:,1] - 10
# #-------------------------------------------------------------------------------------

f = tf.constant([100,100,20]) #fidelity in x, y, z # < 5s
lim = tf.constant([-100.,100.,-100.,100.,-10.,10.])

for _ in range(1):
    E = subdivide_scan_tf(cloud, plt2, bounds = lim, fid = f, draw_grid = False, show_pc = False) 
# ViewInteractiveWidget(plt2.window)
# print(E)

took 0.07901787757873535 seconds with tensorflow


## Unit Test R() and Jacobian

In [ ]:
n_hat = np.array([1,0,0])
theta =  0.1 #np.pi/6 #rad

rot_mat_simp = R_simp(n_hat, theta)
print(rot_mat_simp)
angs = np.array([theta,0 ,0])
rot_mat = R(angs)
print(rot_mat)

print(R2Euler(rot_mat))

p_point = np.array([1,1,1]).T

J = jacobian(angs, p_point)

d_rot_mat_simp = dR_simp(n_hat,theta)
# print(d_rot_mat_simp)

## Unit test R2Euler_tf and R_tf
#### Works with single axis roation
#### Works with vectoried input
#### Solution becomes ambiguious with mutliple axis inputs

In [ ]:
angs = tf.random.normal((3,1)) * tf.constant([[0.], [0.], [1.] ]) #single axis angle input
# angs = tf.Variable([[0.],[ np.pi/2],[ 0.]]) #vector input, multiple axis rotation
print("Input angs: \n", angs.numpy())
# print("R(angs): \n", R_tf(angs).numpy())
test1 = R2Euler_tf(R_tf(angs))
print("R2Euler_tf(R(angs)): \n", test1.numpy())
# test2 = R2Euler(R(angs[:,:2]))
# print(test2)

### Unit test tfp find bins

In [ ]:
c = tf.convert_to_tensor(cloud, np.float32)#[:100]
# print("c: \n", c.numpy())
startx = -100.
stopx = 100.
numx = 10
edgesx = tf.linspace(startx, stopx, numx)
xbins = tfp.stats.find_bins(c[:,0], edgesx)
print(xbins)
starty = -100.
stopy = 100.
numy = 10
edgesy = tf.linspace(starty, stopy, numy)
ybins = tfp.stats.find_bins(c[:,1], edgesy)
print(ybins)

min_num_pts = 1000

count = 0
E = []

for x in range(numx):
    for y in range(numy):
        #only do calculations if there are a sufficicently high number of points in the bin
        xin = tf.where(xbins == x)
        if tf.shape(xin)[0] > min_num_pts:
            if tf.shape(tf.where(tf.gather(ybins, xin) == y))[0] > min_num_pts: #repeat for y points at x coord
#                 print("working", x, y)
                count += 1
# print(xin)
# print(ybins)
print(count)

In [ ]:
test = tf.linspace(1,5,5)
print(test)
ans = tf.where(test < 4)
print("ans: \n",ans)
print("\n", tf.gather(test, ans))


## Inline volumetric rendering using ipyvolume

In [ ]:
#simple demo
import ipyvolume
ds = ipyvolume.datasets.aquariusA2.fetch()
short = ds.data[:,:,:]
ipyvolume.quickvolshow(short, lighting=True)

In [ ]:
ans = np.linspace(1,12,8)
ans[2] = 0
print(ans)

test = ans[ans < 10]
print(test)
np.shape(test)[0]

print(np.median(test[:3]))

In [ ]:
tes = tf.random.normal([3,3])
print(tes)
print(tf.reverse(tes, axis = [1]))

In [ ]:
# a = tf.reshape([10.,10.,2.], (3,1))
# print(type(a))
# t = tf.Tensor(a, dtype = "float32")

a = tf.constant([2.1,2.,3.])
b = tf.constant([1.,2.,3.])
tf.tensordot(a,b, axes = 0)

In [ ]:
eye = tf.eye(3)
# print(j)
Jx = tf.constant([[1.], [2.], [3.]])
J = tf.concat([eye, Jx], axis = 1)
print(J)

In [ ]:
# print(tf.sin(1.))
start = time.time()
numiter = 1000
angs  = np.array([1.,0.1,0.1])
p_point = np.array([1.,2.,3.])
for _ in range(numiter):
    J = jacobian(angs, p_point);
print("took", time.time()-start, "seconds on CPU")
print(J)

In [ ]:
#TF is slower if we do them one by one BUT is waaay faster if we send them in all at once
start = time.time()
numiter = 1000
angs = tf.random.normal((3,numiter))
p_point = tf.random.normal((3,numiter))
J = jacobian_tf(angs, p_point);
print("took", time.time()-start, "seconds on GPU")
print(J.numpy())

### Testing subdividing cells without loop

In [105]:
#Goal: given input tensor "cloud" and "bins" which contains all og and binned coordinates
#      subdivide and perform ops on "cloud" wihtout using any loops 

bins = tf.transpose(tf.constant([[2., 1., 3., 0., 2., 2., 3., 1., 0., 2.],
                                 [0., 1., 0., 0., 0., 0., 3., 1., 0., 0.],
                                 [1., 2., 3., 0., 1., 1., 3., 2., 0., 1.]]))
cloud = bins + tf.random.normal(tf.shape(bins))*0.1
print("binned coordinate values: \n", bins)
# print(cloud)

#1d case (easy)
# q = tf.constant([1., 1., 2.]) 
# print(tf.squeeze(tf.gather(bins, ans))) #works for 1d, unsure of utility in 2d
#2d case (hard)
q = tf.constant([[[1., 1., 2.]],
                 [[0., 9., 9.]],
                 [[2., 0., 1.]],
                 [[7., 8., 9.]]])
print("\n cells of interest: \n",q)

idx = tf.equal(bins, q)
print(idx)
#ans outputs tensor of shape [N,2], where:
#  [[voxel number, index of [x,y,z] in cloud that corresponds to bin #],
#   [voxel number,index of [x,y,z] in cloud that corresponds to bin #]] ... 
loc = tf.where(tf.math.reduce_all(idx, axis = 2) == True)
print("\n loc: \n", loc)

#Need to "ungroup" so that we can fit_gaussian_tf() to each individual voxel...
s = tf.shape(loc)
group_ids, group_idx = tf.unique(loc[:, 0], out_idx=s.dtype)
num_groups = tf.reduce_max(group_idx) + 1
# print(group_ids, group_idx, num_groups)
sizes = tf.math.bincount(group_idx)
# print(sizes)

#replace <bins> here with <cloud> when done debugging
rag = tf.RaggedTensor.from_row_lengths(tf.gather(cloud, loc[:,1]), sizes) 
# print("ragged: \n", rag)

#Run on GPU as vectorized operation (WAAAAAY Faster) --
reg = tf.RaggedTensor.to_tensor(rag)
print("\n regular tensor: \n", reg)
mu, sigma = fit_gaussian_tf(reg)
print("mu: \n", mu)
print("sigma: \n", sigma)
#------------------------------------------------------- 

# # works but uses loop (runs on CPU -> slow) -----------
# A =  tf.data.Dataset.from_tensor_slices(rag)
# mus = []
# sigmas = []
# for i in range(len(A)):
#     mu, sigma = fit_gaussian_tf(rag[i])
#     mus.append(mu)
#     sigmas.append(sigma)
# print(mus, sigmas)
# #------------------------------------------------------

binned coordinate values: 
 tf.Tensor(
[[2. 0. 1.]
 [1. 1. 2.]
 [3. 0. 3.]
 [0. 0. 0.]
 [2. 0. 1.]
 [2. 0. 1.]
 [3. 3. 3.]
 [1. 1. 2.]
 [0. 0. 0.]
 [2. 0. 1.]], shape=(10, 3), dtype=float32)

 cells of interest: 
 tf.Tensor(
[[[1. 1. 2.]]

 [[0. 9. 9.]]

 [[2. 0. 1.]]

 [[7. 8. 9.]]], shape=(4, 1, 3), dtype=float32)
tf.Tensor(
[[[False False False]
  [ True  True  True]
  [False False False]
  [False False False]
  [False False False]
  [False False False]
  [False False False]
  [ True  True  True]
  [False False False]
  [False False False]]

 [[False False False]
  [False False False]
  [False False False]
  [ True False False]
  [False False False]
  [False False False]
  [False False False]
  [False False False]
  [ True False False]
  [False False False]]

 [[ True  True  True]
  [False False False]
  [False  True False]
  [False  True False]
  [ True  True  True]
  [ True  True  True]
  [False False False]
  [False False False]
  [False  True False]
  [ True  True  True]]

 [[Fa

### Find "loc" more efficiently than using tf.where

In [106]:
print("binned points: \n", bins)
print("\nbins to place them: \n",q)

# #SUPER inefficient 
# for i in range(tf.shape(bins)[0]):
#     for j in range(tf.shape(q)[0]):
#         if tf.reduce_all(bins[i] == q[j]):
#             print(j,i)
#             try:
#                 loc2 = tf.concat((loc2, tf.constant([[j,i]])), axis = 0)
#             except:
#                 loc2 = tf.constant([[j,i]])
# print("\n loc \n",loc2)
# loc2 = None

testidx = tf.where(bins == q[1])
print(testidx)
    
print("\n goal is to get this: \n",loc)

binned points: 
 tf.Tensor(
[[2. 0. 1.]
 [1. 1. 2.]
 [3. 0. 3.]
 [0. 0. 0.]
 [2. 0. 1.]
 [2. 0. 1.]
 [3. 3. 3.]
 [1. 1. 2.]
 [0. 0. 0.]
 [2. 0. 1.]], shape=(10, 3), dtype=float32)

bins to place them: 
 tf.Tensor(
[[[1. 1. 2.]]

 [[0. 9. 9.]]

 [[2. 0. 1.]]

 [[7. 8. 9.]]], shape=(4, 1, 3), dtype=float32)
tf.Tensor(
[[3 0]
 [8 0]], shape=(2, 2), dtype=int64)

 goal is to get this: 
 tf.Tensor(
[[0 1]
 [0 7]
 [2 0]
 [2 4]
 [2 5]
 [2 9]], shape=(6, 2), dtype=int64)


### remove zero rows from 3d tensor

In [107]:
bins2 = tf.transpose(tf.constant([[[2., 9., 3., 0., 2., 2., 3., 1., 0., 2.], #x
                                  [0., 0., 0., 0., 0., 0., 3., 1., 0., 0.],
                                  [0., 1., 0., 0., 0., 0., 3., 1., 0., 0.],
                                  [1., 2., 3., 0., 1., 1., 3., 2., 0., 1.]], 
                                  
                                  [[2., 1., 3., 0., 2., 2., 3., 1., 0., 2.], #y
                                  [0., 0., 0., 0., 0., 0., 3., 1., 0., 0.],
                                  [0., 1., 0., 0., 0., 0., 3., 1., 0., 0.],
                                  [1., 2., 3., 0., 1., 1., 3., 2., 0., 1.]],
                                  
                                  [[2., 1., 3., 1., 2., 2., 3., 1., 0., 2.], #z
                                  [0., 0., 0., 0., 0., 0., 3., 1., 0., 0.],
                                  [0., 1., 0., 0., 0., 0., 3., 1., 0., 0.],
                                  [1., 2., 3., 0., 1., 1., 3., 2., 0., 1.]]]))

print(bins2[:2])
print(bins2[:,:,0])
#need to AVERAGE point locations PER AXIS, PER BIN
#   Ignore SPECIFIC POINTS where XYZ are ALL ZERO
idx = tf.math.not_equal(bins2[:,:,0], tf.constant([0.,0.,0.]))
print(idx[:2])
mask = tf.where(tf.math.reduce_any(idx, axis = 2) == True)
print("\n mask: \n", mask[:6]) #correct(?)
# print(tf.gather(bins,mask))

nonzero = tf.gather(bins,[0,0,0])
print("\n nonzero elements: \n", nonzero)

tf.Tensor(
[[[2. 2. 2.]
  [0. 0. 0.]
  [0. 0. 0.]
  [1. 1. 1.]]

 [[9. 1. 1.]
  [0. 0. 0.]
  [1. 1. 1.]
  [2. 2. 2.]]], shape=(2, 4, 3), dtype=float32)
tf.Tensor(
[[2. 0. 0. 1.]
 [9. 0. 1. 2.]
 [3. 0. 0. 3.]
 [0. 0. 0. 0.]
 [2. 0. 0. 1.]
 [2. 0. 0. 1.]
 [3. 3. 3. 3.]
 [1. 1. 1. 2.]
 [0. 0. 0. 0.]
 [2. 0. 0. 1.]], shape=(10, 4), dtype=float32)


InvalidArgumentError: required broadcastable shapes [Op:NotEqual]

### Generate 2D tensor with all permutations (:n1, :n2, :n3)

In [108]:
fida = 3 
fidb = 3
fidc = 3

a = tf.linspace(0,fida-1,fida)[:,None]
b = tf.linspace(0,fidb-1,fidb)[:,None]
c = tf.linspace(0,fidc-1,fidc)[:,None]

ansa = tf.tile(a, [fidb*fidc, 1])
ansb = tf.tile(tf.reshape(tf.tile(b, [1,fida]), [-1,1] ), [(fidc), 1])
ansc = tf.reshape(tf.tile(c, [1,fida*fidb]), [-1,1] )

q = tf.squeeze(tf.transpose(tf.Variable([ansa,ansb,ansc])))
print(q)

#GOAL- determine which voxel pt belongs in based on its coords
# pt = tf.constant([2,1,0])
pt = bins
print("\n pt:", pt, "\n")
num = tf.cast( ( pt[:,0] + fida*pt[:,1] + (fida*fidb)*pt[:,2] ), tf.int32)
print("\n", num, "\n")
# print(q[num])

ans = tf.concat((num[:,None], tf.cast(tf.linspace(0, tf.shape(pt)[0], tf.shape(pt)[0]  )[:,None],
                                      dtype = tf.int32) ), axis = 1 )
print(ans)


tf.Tensor(
[[0. 0. 0.]
 [1. 0. 0.]
 [2. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]
 [2. 1. 0.]
 [0. 2. 0.]
 [1. 2. 0.]
 [2. 2. 0.]
 [0. 0. 1.]
 [1. 0. 1.]
 [2. 0. 1.]
 [0. 1. 1.]
 [1. 1. 1.]
 [2. 1. 1.]
 [0. 2. 1.]
 [1. 2. 1.]
 [2. 2. 1.]
 [0. 0. 2.]
 [1. 0. 2.]
 [2. 0. 2.]
 [0. 1. 2.]
 [1. 1. 2.]
 [2. 1. 2.]
 [0. 2. 2.]
 [1. 2. 2.]
 [2. 2. 2.]], shape=(27, 3), dtype=float64)

 pt: tf.Tensor(
[[2. 0. 1.]
 [1. 1. 2.]
 [3. 0. 3.]
 [0. 0. 0.]
 [2. 0. 1.]
 [2. 0. 1.]
 [3. 3. 3.]
 [1. 1. 2.]
 [0. 0. 0.]
 [2. 0. 1.]], shape=(10, 3), dtype=float32) 


 tf.Tensor([11 22 30  0 11 11 39 22  0 11], shape=(10,), dtype=int32) 

tf.Tensor(
[[11  0]
 [22  1]
 [30  2]
 [ 0  3]
 [11  4]
 [11  5]
 [39  6]
 [22  7]
 [ 0  8]
 [11 10]], shape=(10, 2), dtype=int32)


### Calculate covariance of 3d tensors with multiple voxels. Ignore zeros.

In [113]:
a = tf.random.normal(shape=(10, 2, 1))
b = tf.random.normal(shape=(10, 2, 1))*4
c = tf.random.normal(shape=(10, 2, 1))

d = tf.concat((a,b,c), axis = 2)
d = tf.concat((d, tf.zeros((10,2,3))), axis = 0)
print(d[:,1])

I = tf.sparse.eye(10,3)
print("\n Sparse Identity: \n", I)

cov = tfp.stats.covariance(d, sample_axis = 0, event_axis = 2)
print("\n covariance matrices: \n", cov[1])

print(sizes)


tf.Tensor(
[[-0.8948956   2.2362711  -0.43921176]
 [-0.35083345 -3.1048205  -0.10731926]
 [-0.5755011  -2.7571492   0.9450426 ]
 [ 0.8119266  -3.7354808   1.0391369 ]
 [-0.46134612  4.5810437   0.01974574]
 [-1.6642511   0.83993524  0.26953086]
 [ 0.19249603  6.0262723   0.7527277 ]
 [-1.1216501  -1.4189025  -0.02843553]
 [-1.6132956   0.6021511  -1.7156568 ]
 [-0.24320705 -3.1648061  -0.13676174]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]], shape=(20, 3), dtype=float32)

 Sparse Identity: 
 SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 1]
 [2 2]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32), dense_shape=tf.T